In [ ]:
#default_exp animal_classes

# Ecosystems

## Google Colab preparations

In [ ]:
#export
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

IN_MAIN = __name__ == '__main__'

In [ ]:
#Infrastructure for copying notebooks
if IN_COLAB and IN_MAIN:
    home_dir = '/content/drive/MyDrive/Colab Notebooks/Ecosystems/v3'
if IN_COLAB and IN_MAIN:
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append(home_dir)
    %cd $home_dir
    
!pip -q install import-ipynb

## Modules

In [ ]:
#export
# Standard modules
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from gym import spaces

# import import_ipynb

In [ ]:
#export
# Abstract class modules
from abc import ABC, abstractmethod 
from dataclasses import dataclass
from dataclasses import field

#The module typing provides runtime support for type hints. 
#The most fundamental support consists of the types Any, Union, Tuple, Callable, TypeVar, and Generic.
from typing import Any, Union, List, Set, Dict, Tuple, Optional, Callable

# # Use Union when something could be one of a few types
# x: List[Union[int, str]] = [3, 5, 'test', 'fun']

# # Use Any if you don't know the type of something or it's too
# # dynamic to write a type for
# x: Any = mystery_function()


In [ ]:
#export
# Homemade modules
from ecotwins.utility import normalize, distance, draw_objects, motion_diagram
from ecotwins.perception import Perception

In [ ]:
#export
_N_OBJECTS = 10
def generate_objects(side=None, n_objects=_N_OBJECTS):
    """Generates a random map of objects 
    param side: size of each side of the grid
    param n_objects: (initial) number of objects in the ecosystem
    """
    objects=(np.random.rand(n_objects,2)-0.5)*side # Takes values in (-mothenv.SIDE/2,+mothenv.SIDE/2)
    return objects

## Terrain

In [ ]:
#export
@dataclass
class Terrain:
    space: Tuple[List] = field(default_factory=lambda: np.array(([-10, 10], [-10, 10])))
    conditions: Callable[[List[int]], Dict[str, int]] = field(
        default_factory=lambda: lambda pos: {"temp": 11, "wind": 2}
    )
    objects: Dict[str, List[Any]] = field(
        default_factory=lambda: {"rock": [(5, 5), (4, 4)]}
    )
    counter: int = 0
    # currently only torus convention while moving, not in is_close and consume_closest_object
    torus: bool = False
        
    def __post_init__(self):
        objects = self._build_objects(self.objects)
        self.original_objects = self.objects
        self.objects = objects
        # Keep track of the objects that are consumed. Mainly
        # for plotting and statistics
        self.consumed = {k:[] for k in objects.keys()}
        
        if self.torus: # Will be used in the compuations
            self.left = self.space[:, 0]
            self.side = self.space[:, 1] - self.space[:, 0]
        
        
    def _build_objects(self, orig_objects):
        # Construct the actual object list.
        actual_objects = {}
        for name, o in orig_objects.items():
            if isinstance(o, int): 
                # Generate o objects of the specified type
                # This does only work if the self.space is square and ranges along
                # the x and y axis are the same.
                objects = generate_objects(side=self.space[0, 1] - self.space[0, 0], n_objects=o)
            else: 
                objects = np.array(o)
            actual_objects[name] = objects
            
        return actual_objects
    
    def move(self, pos, step):
        # Probably not the intended stop. One would 
        # prefer to stop along the step vector.
        # Maybe should the terrain be aware of the agent that wants to move
        # and an agent(animal) should be used as argument instead of pos.
#         print(__name__, pos, step)
        new_pos = pos + step
        if self.torus:
            new_pos = (new_pos - self.left) % self.side + self.left
        else:
            new_pos[0] = np.clip(new_pos[0], *self.space[0])
            new_pos[1] = np.clip(new_pos[1], *self.space[1])
        return new_pos
    
    def update(self):
        counter += 1
        pass
    
    def is_close(self, pos, object_type, threshold):
        if isinstance(object_type, str):
            object_type = [object_type]
            
        for name in object_type:
            d = distance(pos, self.objects[name])

            # if distance(pos, self.objects[name]).min() < threshold:
            if (d.shape[0] != 0) and (d.min() < threshold):
                return True

        return False
    
    def _consume_object(self, name, idx):
        o = self.objects[name]
        assert idx < len(o)
        
        self.consumed[name].append(o[idx].copy())
        if isinstance(self.original_objects[name], int):
            # respawn
            o[idx] = generate_objects(side=self.space[0, 1] - self.space[0, 0], n_objects=1)
        else:
            # remove
            self.objects[name] = np.concatenate((o[:idx], o[idx+1:]))
        
    
    def consume_closest_object(self, position, name):
        o = self.objects[name]
#         print(f'{position}\n distance{distance(position, o)}')
        idx = distance(position, o).argmin()
        self._consume_object(name, idx)
        
        
    # A more general version of consume_closest_object that supports a list of
    # names as well.
    def consume_closest_object(self, position, names):
        names = [names] if isinstance(names, str) else names
         
        closest, idx, c_name = np.inf, -1, ""
        for s, o, dist in (
            (s, self.objects[s], distance(position, self.objects[s]))
            for s in names
        ):
            # print(dist.shape)
            if dist.shape[0] == 0: # No objects
                continue
            if dist.min() < closest:
                closest, idx, c_name = dist.min(), dist.argmin(), s
            
        if c_name != "": self._consume_object(c_name, idx)
        
        return c_name
    

    def is_done(self):
        """ Currently very simple, done if there are not objects. """
        return sum(list(map(lambda x: len(x), self.objects.values()))) == 0
    
    
    def reset(self):
        self.objects = self._build_objects(self.original_objects)
        self.counter = 0
        self.consumed = {k:[] for k in self.objects.keys()}
        
if IN_MAIN:
    delsjön = Terrain()
    delsjön.conditions([2, 2])["temp"]
    delsjön.move(np.array([1,1]), np.array([20,2]))

## Organism

In [ ]:
#export
@dataclass
class Organism(ABC): 

    species: str
    genome: str 
    sex: str 
    age: float 
    weight: float
    position: Any 
    hyperparameters: Dict[str,float]
    homeostasis: Dict[str,float]
    min_homeostasis: Dict[str,float] 
    max_homeostasis: Dict[str,float]
    homeostatic_effects: Dict[str,float] # No actions in plants. Effects depend on temp, rain, light,...
   
#   super().__init__()

# Can we turn alive into a concrete method as in Dandelion?
    @abstractmethod
    def alive(self): # Depends on homeostasis and age
        pass


    def __eq__(self, org):
        """Somewhat ugly, for now considered equal if they are in the same position."""
        return (type(self) == type(org)) and (self.position == org.position).all()


### Plants

In [ ]:
@dataclass
class Plant(Organism): 
     pass

#### Dandelion

In [ ]:
# Dict for giving default values to dandelions
dandelion_def = {
    'species': 'dandelion',
    'genome': '',
    'sex': 'neutral',
    'age': 0,
    'weight': 22,
    'position': lambda: np.array([0.0, 0.0]),
    'homeostasis': lambda: {
        'energy': 3, 
        'water': 2, 
        'temp': 37,
    },
    'min_homeostasis': lambda: {
        'energy':0,
        'water':0,
        'temp': 0,
    },
    'max_homeostasis': lambda: {
        'energy':10,
        'water':10,
        'temp': 50,
    },
    'homeostatic_effects': lambda: {
        ('idle','water'): {'energy': 0, 'water': 1},
    },
    'hyperparameters': lambda: {
        # Biological
        'max_age': 100, 
        # Physical
        'close': 0.01, # distance for mating
    },
} 

In [ ]:
 @dataclass
class Dandelion(Plant):
    species: str = dandelion_def['species']
    genome: str = dandelion_def['genome']
    sex: str = dandelion_def['sex']
    age: float = dandelion_def['age']
    weight: float = dandelion_def['weight']
    position: np.ndarray = field(default_factory=dandelion_def['position'])
    hyperparameters: Dict[str,float] = field(default_factory=dandelion_def['hyperparameters'])
    homeostasis: Dict[str,float] = field(default_factory=dandelion_def['homeostasis'])
    min_homeostasis: Dict[str,float] = field(default_factory=dandelion_def['min_homeostasis'])
    max_homeostasis: Dict[str,float] = field(default_factory=dandelion_def['max_homeostasis'])
    homeostatic_effects: Dict[str,float] = field(default_factory=dandelion_def['homeostatic_effects'])
   
    def alive(self): 
        '''It is alive if the water and energy levels are ok and it is not too old'''
        return self.homeostasis['energy'] > self.min_homeostasis['energy'] and \
            self.homeostasis['water'] > self.min_homeostasis['water'] and \
            self.homeostasis['temp'] > self.min_homeostasis['temp'] and \
            self.max_homeostasis['temp'] > self.homeostasis['temp'] and \
            self.age < self.hyperparameters['max_age']

In [ ]:
if IN_MAIN:
    dandelion1 = Dandelion()
    dandelion1.alive()

### Animals

In [ ]:
#export
@dataclass(eq=False)
class Animal(Organism): 
    # Everything from Organism + these (relating to perception and actions):
    distances: Dict # Perception radii for different objects
    perception: Dict
    interoception: Dict
    exteroception: Dict
    proprioception: Any # Use union for vector + 'eat'?
    object_perception: Any
 
    @abstractmethod
    def happiness(self): # Depends on perception
        pass

    # Compute the next object_perception from the next positions of the objects
    # Only those objects that are unside the radius
    @abstractmethod
    def object_perception(self, ecosystem): 
        pass

Consider adding a nervous system (or 'brain') with 
* a policy network 
* a reflex network 
* a prediction network.

#### Sheep

In [ ]:
# Dict for giving default values to sheep

sheep_radii = {
    'wolf': 10, 
    'dandelion': 2,
    }
sheep_def = {
    'species': 'sheep',
    'genome': '',
    'sex': 'male',
    'age': 3,
    'weight': 22,
    'position': lambda: np.array([0.0, 0.0]),
    'homeostasis': lambda: {
        'energy': 3, 
        'water': 2, 
        'temp': 37,
        'hormones': 10,
    },
    'min_homeostasis': lambda: {
        'energy':10,
        'water':10,
        'temp': 30,
        'hormones': 0,
    },
    'max_homeostasis': lambda: {
        'energy':10,
        'water':10,
        'temp': 40,
        'hormones': 10,
    },
    'homeostatic_effects': lambda: { # Nutrition table
        ('eat', 'dandelion'): {'energy': 1, 'water': 0.2},
        ('drink','water'): {'energy': 0, 'water': 1},
    },
    'hyperparameters': lambda: {
        # Biological
        'max_age': 100, 
        # Physical
        'delta': 0.01, # stride max_inclination, max_speed (max_values)
        'close': 0.01, # distance for eating, drinking, attacking, mating
        # Metabolism: 
        'energy_cost':0.01, 
        'water_cost':0.02,
        # Psychological
        'gamma':0.9,
    },
    # 'distances': lambda: {
    #     'wolf': 10, 
    #     'dandelion': 2,
    'distances': lambda: sheep_radii,

    # Till defaultfactoryn (generera en dictionary):
    'perception': lambda: {key:Perception(radius) for (key,radius) in sheep_radii.items()},

    'interoception': lambda: {
        'energy': 3, 
        'water': 2,
    }, 
    'exteroception': lambda: {
        'temp': 10,
        'humidity': 88,
    }, 
    'proprioception': 'eat',  # (0.2,0,9) # move direction + speed or 'eat' or 'mate' or 'idle'
    'object_perception': lambda: {
        'wolf': (1, 2, 10.0),
        'dandelion': (3, 3, 2.0),
        'sheep': (3, 3, 2.0),
    },
} 

In [ ]:
@dataclass(eq=False)
class Sheep(Animal):
    species: str = sheep_def['species']
    genome: str = sheep_def['genome']
    sex: str = sheep_def['sex']
    age: float = sheep_def['age']
    weight: float = sheep_def['weight']
    position: np.ndarray = field(default_factory=sheep_def['position'])
    hyperparameters: Dict[str,float] = field(default_factory=sheep_def['hyperparameters'])
    homeostasis: Dict[str,float] = field(default_factory=sheep_def['homeostasis'])
    min_homeostasis: Dict[str,float] = field(default_factory=sheep_def['min_homeostasis'])
    max_homeostasis: Dict[str,float] = field(default_factory=sheep_def['max_homeostasis'])
    homeostatic_effects: Dict[str,float] = field(default_factory=sheep_def['homeostatic_effects'])
    distances: Dict[str, float] = field(default_factory=sheep_def['distances'])
    perception: Dict[str, Perception] = field(default_factory=sheep_def['perception'])
    interoception: Dict[str, float] = field(default_factory=sheep_def['interoception'])
    exteroception: Dict[str, float] = field(default_factory=sheep_def['exteroception'])
    proprioception: str = sheep_def['proprioception']
    object_perception: Dict[str, float] = field(default_factory=sheep_def['object_perception'])
    action: spaces.Box = field(default_factory=lambda: spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float16))

    def alive(self): # Depends on homeostasis
        return self.homeostasis['energy'] > 0 and self.homeostasis['water'] > 0 and self.age < self.hyperparameters['max_age']
    
    def happiness(self): # Depends on perception like energy (interoception), humidity (exteroception), and smell (object_perception)
        glucose = self.interoception['energy']
        return glucose
    
    def happiness2(self):
        glucose = self.interoception['energy']
        water = self.interoception['water']
        return glucose*water

    def happiness3(self):
        glucose = self.interoception['energy']
        smell = self.object_perception['sheep'][2]
        return glucose*(0.5+0.5*smell)

    def update(self):
        self.position = terrain.move(self.position, self.action)


#### SimpleSheep

In [ ]:
#export
# Dict for giving default values to simple_sheep

simple_sheep_radii = {
    'dandelion': 2,
    }
simple_sheep_def = {
    'species': 'simple_sheep',
    'genome': '',
    'sex': 'male',
    'age': 0,
    'weight': 22,
    'position': lambda: np.array([0.0, 0.0]),
    'homeostasis': lambda: {
        'smell': 0, 
    },
    'min_homeostasis': lambda: {
        'smell':-1,
    },
    'max_homeostasis': lambda: {
        'smell':10,
    },
    'homeostatic_effects': lambda: { # Nutrition table
        ('move', None): {'energy': -0.01, 'water': -0.01},
        ('consume', 'water'): {'energy': 0, 'water': 1},
        ('consume', 'dandelion'): {'energy': 1, 'water': 0.1},
    },
    'hyperparameters': lambda: {
        # Biological
        'max_age': 2000, 
        # Physical
        'delta': 0.01, # stride max_inclination, max_speed (max_values)
        'close': 0.1, # distance for eating, drinking, attacking, mating
        # Psychological
        'gamma':0.9,
    },
    'distances': lambda: simple_sheep_radii,

    # Till defaultfactoryn (generera en dictionary):
    'perception': lambda: {key:Perception(radius) for (key,radius) in simple_sheep_radii.items()},

    'interoception': lambda: {
        'energy': 3, 
        'water': 2,
    }, 
    'exteroception': lambda: {
        'temp': 10,
        'humidity': 88,
    }, 
    'proprioception': 'eat',  # (0.2,0,9) # move direction + speed or 'eat' or 'mate' or 'idle'
    'object_perception': lambda: {
        'dandelion': (3, 3, 2.0),
    },
} 

In [ ]:
#export
"""A simple class that moves in the smell direction and the objects is consumed
   when the agent is close enough."""
@dataclass(eq=False)
class SimpleSheep(Animal):
    species: str = simple_sheep_def['species']
    genome: str = simple_sheep_def['genome']
    sex: str = simple_sheep_def['sex']
    age: float = simple_sheep_def['age']
    weight: float = simple_sheep_def['weight']
    position: np.ndarray = field(default_factory=simple_sheep_def['position'])
    hyperparameters: Dict[str,float] = field(default_factory=simple_sheep_def['hyperparameters'])
    homeostasis: Dict[str,float] = field(default_factory=simple_sheep_def['homeostasis'])
    min_homeostasis: Dict[str,float] = field(default_factory=simple_sheep_def['min_homeostasis'])
    max_homeostasis: Dict[str,float] = field(default_factory=simple_sheep_def['max_homeostasis'])
    homeostatic_effects: Dict[str,float] = field(default_factory=simple_sheep_def['homeostatic_effects'])
    distances: Dict[str, float] = field(default_factory=simple_sheep_def['distances'])
    perception: Dict[str, Perception] = field(init=False)
    interoception: Dict[str, float] = field(default_factory=simple_sheep_def['interoception'])
    exteroception: Dict[str, float] = field(default_factory=simple_sheep_def['exteroception'])
    proprioception: str = simple_sheep_def['proprioception']
    object_perception: Dict[str, float] = field(default_factory=simple_sheep_def['object_perception'])
    # Different to MultiSheep:
    action_space: spaces.Box = field(default_factory=lambda: spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32))
    observation_space: spaces.Box = field(default_factory=lambda: spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32))
    use_reward_radius_in_perception: bool = False
#     action: np.ndarray = 

    
    def __post_init__(self):
        rr = self.hyperparameters['close'] if self.use_reward_radius_in_perception else 0
        self.perception = {key:Perception(radius, reward_radius=rr) for (key,radius) in self.distances.items()}
        self.p_happiness = 0

    def alive(self): # Depends on homeostasis
        return True
    
    # Problematic version corresponding to pure taxis
    def happiness_problematic(self, objects: Dict[str, List]):
        # This sheep can only observe one sort of objects
        assert len(self.perception) == 1
        name, p = next(iter(self.perception.items()))
        o = objects[name]
        h = p.total_intensity(self.position, o)
        
        return h

    # Version corresponding to pure taxis
    def happiness(self, terrain: Terrain):
        assert len(self.perception) == 1
        name, p = next(iter(self.perception.items()))
        # h = terrain.is_close(self.position, name, self.hyperparameters['close'])
        
        return self.p_happiness
    
    # Version that hopefully corresponds to World 2 (a combination)
    def happiness(self, terrain: Terrain):
        assert len(self.perception) == 1
        name, p = next(iter(self.perception.items()))
        h = terrain.is_close(self.position, name, self.hyperparameters['close'])
        
        return float(h) + self.p_happiness
        
    
    def update(self, action, terrain):
        assert len(self.perception) == 1
        name, p = next(iter(self.perception.items()))
        o = terrain.objects[name]
        self.action = action
        self.p_happiness = self.happiness(terrain)
        if terrain.is_close(self.position, name, self.hyperparameters['close']): # We are close to an eatable object, eat it
            terrain.consume_closest_object(self.position, name)
            
        self.position = terrain.move(self.position, self.hyperparameters['delta'] * action)
        self.age += 1 
        
    
    def observation(self, terrain: Terrain):
        assert len(self.perception) == 1
        name, p = next(iter(self.perception.items()))
        o = terrain.objects[name]
        return p.multi_direction(self.position, o)
    
    def is_done(self):
        return self.age > self.hyperparameters['max_age']
    
    def out_of_resources(self):
        return False # Never runs out.
        
        
    def reset(self):
        # This might not be what one wants for instance if one supplies
        # other intial values to age and position.
        self.age = simple_sheep_def['age']
        self.position = simple_sheep_def['position']()
        self.p_happiness = 0

In [ ]:
if IN_MAIN:
    shaun = Sheep()
    shaun2 = Sheep(position=(1,1))
    shaun.distances['wolf'] = 100
    shaun.alive()#happiness()
    shaun, shaun2

    shaun2 = Sheep(proprioception='eat')
    shaun2.position[:] = np.random.random(2)
    shaun2.proprioception = 'gamble'
    shaun2.proprioception

#### MultiSheep
A sheep with multiple needs (water energy)

In [ ]:
def generate_objects(side=None, n_objects=N_OBJECTS):
    """Generates a random map of objects 
    param side: size of each side of the grid
    param n_objects: (initial) number of objects in the ecosystem
    """
    objects=(np.random.rand(n_objects,2)-0.5)*side # Takes values in (-mothenv.SIDE/2,+mothenv.SIDE/2)
    return objects

In [ ]:
#export
# Dict for giving default values to multi_sheep

multi_sheep_radii = {
    'dandelion': 2,
    'water': 2,
    }
multi_sheep_def = {
    'species': 'multi_sheep',
    'genome': '',
    'sex': 'male',
    'age': 0,
    'weight': 22,
    'position': lambda: np.array([0.0, 0.0]),
    'homeostasis': lambda: {
        'smell': 0, 
    },
    'min_homeostasis': lambda: {
        'smell':-1,
    },
    'max_homeostasis': lambda: {
        'smell':10,
    },
    'homeostatic_effects': lambda: { # Nutrition table
        ('move', None): {'energy': -0.01, 'water': -0.01},
        ('consume', 'water'): {'energy': 0, 'water': 1},
        ('consume', 'dandelion'): {'energy': 1, 'water': 0.1},
    },
    'hyperparameters': lambda: {
        # Biological
        'max_age': 2000, 
        # Physical
        'delta': 0.01, # stride max_inclination, max_speed (max_values)
        'close': 0.1, # distance for eating, drinking, attacking, mating
        # Psychological
        'gamma':0.9,
    },
    'distances': lambda: multi_sheep_radii,

    # Till defaultfactoryn (generera en dictionary):
    'perception': lambda: {key:Perception(radius) for (key,radius) in multi_sheep_radii.items()},

    'interoception': lambda: {
        'energy': 3, 
        'water': 2,
    }, 
    'exteroception': lambda: {
        'temp': 10,
        'humidity': 88,
    }, 
    'proprioception': 'eat',  # (0.2,0,9) # move direction + speed or 'eat' or 'mate' or 'idle'
    'object_perception': lambda: {
        'dandelion': (3, 3, 2.0),
    },
} 

In [ ]:
#export
"""A simple class that moves in the smell direction and the objects is consumed
   when the agent is close enough. The difference from SimpleSheep is that multiple
   needs are supported and happiness is based on interoception levels."""
@dataclass(eq=False)
class MultiSheep(Animal):
    species: str = multi_sheep_def['species']
    genome: str = multi_sheep_def['genome']
    sex: str = multi_sheep_def['sex']
    age: float = multi_sheep_def['age']
    weight: float = multi_sheep_def['weight']
    position: np.ndarray = field(default_factory=multi_sheep_def['position'])
    hyperparameters: Dict[str,float] = field(default_factory=multi_sheep_def['hyperparameters'])
    homeostasis: Dict[str,float] = field(default_factory=multi_sheep_def['homeostasis'])
    min_homeostasis: Dict[str,float] = field(default_factory=multi_sheep_def['min_homeostasis'])
    max_homeostasis: Dict[str,float] = field(default_factory=multi_sheep_def['max_homeostasis'])
    homeostatic_effects: Dict[str,float] = field(default_factory=multi_sheep_def['homeostatic_effects'])
    distances: Dict[str, float] = field(default_factory=multi_sheep_def['distances'])
    perception: Dict[str, Perception] = field(init=False)
    interoception: Dict[str, float] = field(default_factory=multi_sheep_def['interoception'])
    exteroception: Dict[str, float] = field(default_factory=multi_sheep_def['exteroception'])
    proprioception: str = multi_sheep_def['proprioception']
    object_perception: Dict[str, float] = field(default_factory=multi_sheep_def['object_perception'])
    # action_space: spaces.Box = field(default_factory=lambda: spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32))
    action_space: spaces.Box = field(default_factory=lambda: spaces.Box(low=-np.inf, high=np.inf, shape=(2,), dtype=np.float32))
    observation_space: spaces.Box = field(init=False)
    use_interoception_as_obs: bool = False
    use_intensity_as_obs: bool = False
    use_happiness_as_obs: bool = False
    use_reward_radius_in_perception: bool = False
    use_multi_direction: bool = True
    use_single_direction: bool = False
    action_noise: float = 0.0 # Add a noise vector with components in [-action_noise, action_noise]
    normalize_action: bool = True
    perception_noise: float = 0
    n_frames: int = 1
    
    def __post_init__(self):
        rr = self.hyperparameters['close'] if self.use_reward_radius_in_perception else 0
        self.perception = {key:Perception(radius, reward_radius=rr, direction_noise=self.perception_noise) for (key,radius) in self.distances.items()}
        self.consumables = self._get_consumables()
        self.original_interoception = self.interoception.copy() # Needed for reset
        self.original_position = self.position.copy() # Needed for reset
        self.original_age = self.age

        assert self.use_single_direction or self.use_multi_direction, (
            "One of use_single_direction and use_multi_direction must be True.")
        self.observation_space = self._compute_observation_space()
        self.frame_size = self.observation_space.shape[0] // self.n_frames
        self.p_happiness = self._init_p_happiness() # is used when not using accumulated happiness
        
        self.observation_frame = None # This will be initialized after the first call to `observation`

        
    def _init_p_happiness(self):
        return self.happiness()
    
    def _compute_observation_space(self):
        """Observation space consists of three parts, direction vectors of the
        perceived objects, the corresponding intensity of the perception and the
        internal signal, such as energy level."""
        low, high = [], []
        n_objects = len(self.perception)

        if self.use_multi_direction and self.use_single_direction:
            n_dir = 2
        else:
            n_dir = 1

        # direction vectors, range from -1 to 1
        low.extend([-1] * n_objects * 2 * n_dir)
        high.extend([1] * n_objects * 2 * n_dir)

        # intensity, range from 0 to infinity
        if self.use_intensity_as_obs:
            low.extend([0] * n_objects * n_dir)
            high.extend([np.inf] * n_objects * n_dir)

        # interoception
        if self.use_interoception_as_obs:
            low.extend([0] * len(self.interoception))
            high.extend([np.inf] * len(self.interoception))
            
        # happiness
        if self.use_happiness_as_obs:
            low.append(0)
            high.append(np.inf)

        # Support for many frames the newest fram will be first in order 
        # to keep backwards compatability with reflex agents
        low = low * self.n_frames
        high = high * self.n_frames

        return spaces.Box(low=np.array(low), high=np.array(high), dtype=np.float32)

    # @TODO - Update this for MultiSheep
    def alive(self): # Depends on homeostasis
        return self.happiness() > 0
    
    
    # terrrain argument is present in order not to break
    # MothGym.
    def happiness(self, terrain: Terrain=None):
        """Happiness is the product of the interoception levels."""
        h = 1.0
        for v in self.interoception.values():
            h *= v
        return h 
        
    def _get_consumables(self):
        c = []
        for (action, name) in self.homeostatic_effects.keys():
            if action == 'consume': c.append(name)
        return c
        
        
    # This update is different than for SimpleSheep since energy and water updates
    # are done after the move in the new position. Consumption of objects is also
    # done in the new position.
    def update(self, action, terrain):
        # adjust the observation frame ie rotate it.
        if self.n_frames > 1:
            self.observation(terrain) # To ensure observation_frame in initialized
            self.observation_frame = np.roll(self.observation_frame, self.frame_size)
    
        self.p_happiness = self.happiness()

        # print('action', action, action.shape)
        if self.normalize_action:
            action = normalize(action)
            action += self.action_noise * 2 * (np.random.random(action.shape) - 0.5)
        else:
            d = distance(action, 0)
            if d == 0: d = 1
            else: d = math.tanh(d) / d
            action = d * action

        # print('action', action, action.shape)
        self.position = terrain.move(self.position, self.hyperparameters['delta'] * action)
        self.age += 1
        
        # decrease energy and water levels
        for name, change in self.homeostatic_effects[('move', None)].items():
            assert self.interoception[name] >= 0, 'Negative interoception values'
            self.interoception[name] += change
            
        if terrain.is_close(self.position, self.consumables, self.hyperparameters['close']):
            name = terrain.consume_closest_object(self.position, self.consumables)
            for energy_type, v in self.homeostatic_effects[('consume', name)].items():
                self.interoception[energy_type] += v
                
    def observation(self, terrain: Terrain):
            if self.n_frames == 1:
                return self._one_observation_frame(terrain)
            elif self.observation_frame is None:
                # Initialize observation_frame
                self.observation_frame = np.tile(self._one_observation_frame(terrain), self.n_frames)
            else:
                self.observation_frame[:self.frame_size] = self._one_observation_frame(terrain)
                
            return self.observation_frame.copy()
                
    def _one_observation_frame(self, terrain: Terrain):
        o = []

        if self.use_multi_direction:
            for name, p in self.perception.items():
                o.append(p.multi_direction(self.position, terrain.objects[name]))
            # print('multi', o[-1].shape)
        if self.use_single_direction:
            for name, p in self.perception.items():
                o.append(p.closest_direction(self.position, terrain.objects[name]))
            # print('single', o[-1].shape)
            
        if self.use_intensity_as_obs:
            if self.use_multi_direction:
                for name, p in self.perception.items():
                    o.append(p.total_intensity(self.position, terrain.objects[name]).reshape((1,)))
                # print('multi intensity', o[-1].shape)
            if self.use_single_direction:
                for name, p in self.perception.items():
                    o.append(p.nearest_intensity(self.position, terrain.objects[name]).reshape((1,)))
                # print('single intensity', o[-1].shape)

        if self.use_interoception_as_obs:
            o.append([*self.interoception.values()])
        
        if self.use_happiness_as_obs:
            o.append([self.happiness()])

        return np.concatenate(o)
    
    
    def is_done(self):
        return self.age > self.hyperparameters['max_age'] or self.out_of_resources()
    
    def out_of_resources(self):
        # True if close to or less than 0.
        return any(map(lambda x: math.isclose(x, 0, abs_tol=1e-5) or x < 0, self.interoception.values()))
        
        
    def reset(self):
        # This might not be what one wants for instance if one supplies
        # other intial values to age and position.
        self.age = self.original_age
        self.position = self.original_position.copy()
        self.interoception = self.original_interoception.copy()
        self.p_happiness = self._init_p_happiness()
        if self.n_frames > 1:
            self.observation_frame = None

In [ ]:
if IN_MAIN:
    dolly = MultiSheep(distances={'water':10, 'dandelion':10})
    dolly.interoception
    t = Terrain(objects={'water':10, 'dandelion':10})
#     e = Ecosystem(dolly, t)
    dolly.happiness(t)
    dolly.update(np.array([1,1]), t)

## Ecosystem

In [ ]:
# ampersand dataclass
# class Population:
#     animals: List[Animal]
#     plants: List[Plant]

In [ ]:
#export
class Ecosystem:
    # Current only one organism is supported and it is named 'agent'.
    def __init__(self, terrain: Terrain, agent: Animal):
    # def __init__(self, terrain: Terrain, organisms: List[Organism]):
        self.terrain = terrain
        self.agent = agent
               
    def update(self, agent):
        self.agent.update()
        self.terrain.update() # Bara tidsrelaterade uppdateringar, temp osv
        # koll om terrängen behöver uppdateras

    def is_done(self):
        return self.agent.is_done() or self.terrain.is_done()
               
    def reset(self):
        self.agent.reset()
        self.terrain.reset()
        
        
if IN_MAIN:
    organisms = [shaun,shaun2,dandelion1]
    # eco = Ecosystem(terrain=delsjön,organisms=organisms)
    eco = Ecosystem(terrain=delsjön,agent=organisms[0])

## Ecosystem updates

In [ ]:
# Exteroception (assuming correct perception) 

def exteroception(animal,ecosystem):
    return ecosystem.terrain.conditions(animal.position)

if IN_MAIN: exteroception(shaun,eco)

In [ ]:
# Proprioception (assuming correct perception) 

def proprioception(animal,ecosystem):
    # Dummy. Should return the last action
    return animal.proprioception

if IN_MAIN: proprioception(shaun,eco)

In [ ]:
def homeostasis(animal, ecosystem):
    new = animal.homeostasis
    # Independent of actions
    new["temp"] = exteroception(animal, ecosystem)["temp"]  # snake
    new["hormones"] = min(new["hormones"] + 0.01, animal.max_homeostasis["hormones"])

    # Dependent of actions
    action = animal.proprioception

    ## Independent of close objects
    if np.array(action).shape == (3,):  # when the action is 'move'
        new["energy"] -= animal.hyperparameters[
            "energy_cost"
        ]  # 0.01 # multiply with speed?
        new["water"] -= animal.hyperparameters["water_cost"]  # multiply with speed?

    ## Dependent of close objects
    # for organism in ecosystem.organisms:  # Go through the organisms
    for organism in [ecosystem.agent]:  # Go through the organisms
        print("organism", organism, "animal", animal)
        if (
            organism != animal
            and distance(np.array(animal.position), np.array(organism.position))
            <= animal.hyperparameters["close"]
        ):
            if action == "eat":  # or action == 'drink' or action == 'consume':
                for key in animal.homeostatic_effects:  # Go through the nutrients
                    if key == ("eat", organism.species):
                        for nutrient in animal.homeostatic_effects[key]:
                            new[nutrient] += animal.homeostatic_effects[key][
                                nutrient
                            ]  # Add nutrients. hormones??
                            new[nutrient] = min(
                                new[nutrient], animal.max_homeostasis[nutrient]
                            )
            # How do we handle the 'eat air' case?
            if (
                action == "mate"
                and animal.type == object.type
                and animal.sex != object.sex
            ):
                new["hormones"] = 0  # Lower the hormone level
            if action == "attack":
                pass  # Lower homeostasis of the other animal (too)
    return new


In [ ]:
# shaun moves and loses some energy and water

if IN_MAIN: homeostasis(shaun,eco)

organism Sheep(species='sheep', genome='', sex='male', age=3, weight=22, position=array([0., 0.]), hyperparameters={'max_age': 100, 'delta': 0.01, 'close': 0.01, 'energy_cost': 0.01, 'water_cost': 0.02, 'gamma': 0.9}, homeostasis={'energy': 3, 'water': 2, 'temp': 11, 'hormones': 10}, min_homeostasis={'energy': 10, 'water': 10, 'temp': 30, 'hormones': 0}, max_homeostasis={'energy': 10, 'water': 10, 'temp': 40, 'hormones': 10}, homeostatic_effects={('eat', 'dandelion'): {'energy': 1, 'water': 0.2}, ('drink', 'water'): {'energy': 0, 'water': 1}}, distances={'wolf': 100, 'dandelion': 2}, perception={'wolf': Perception(radius=10, origin=array([0., 0.]), steven_exponent=0.6, reward_radius=0.0, epsilon=0.31622776601683794, use_clip=False, direction_noise=0), 'dandelion': Perception(radius=2, origin=array([0., 0.]), steven_exponent=0.6, reward_radius=0.0, epsilon=0.31622776601683794, use_clip=False, direction_noise=0)}, interoception={'energy': 3, 'water': 2}, exteroception={'temp': 10, 'humid

In [ ]:
# shaun2 is near a dandelion and has action eat

if IN_MAIN: homeostasis(shaun2,eco)

organism Sheep(species='sheep', genome='', sex='male', age=3, weight=22, position=array([0., 0.]), hyperparameters={'max_age': 100, 'delta': 0.01, 'close': 0.01, 'energy_cost': 0.01, 'water_cost': 0.02, 'gamma': 0.9}, homeostasis={'energy': 3, 'water': 2, 'temp': 11, 'hormones': 10}, min_homeostasis={'energy': 10, 'water': 10, 'temp': 30, 'hormones': 0}, max_homeostasis={'energy': 10, 'water': 10, 'temp': 40, 'hormones': 10}, homeostatic_effects={('eat', 'dandelion'): {'energy': 1, 'water': 0.2}, ('drink', 'water'): {'energy': 0, 'water': 1}}, distances={'wolf': 100, 'dandelion': 2}, perception={'wolf': Perception(radius=10, origin=array([0., 0.]), steven_exponent=0.6, reward_radius=0.0, epsilon=0.31622776601683794, use_clip=False, direction_noise=0), 'dandelion': Perception(radius=2, origin=array([0., 0.]), steven_exponent=0.6, reward_radius=0.0, epsilon=0.31622776601683794, use_clip=False, direction_noise=0)}, interoception={'energy': 3, 'water': 2}, exteroception={'temp': 10, 'humid

In [ ]:
# Interoception (assuming correct perception of homeostasis). Could be a subset of homeostasis. Maybe lack sensor for temperature.

def interoception(animal,ecosystem):
    return homeostasis(animal,ecosystem)

if IN_MAIN: interoception(shaun,eco)

organism Sheep(species='sheep', genome='', sex='male', age=3, weight=22, position=array([0., 0.]), hyperparameters={'max_age': 100, 'delta': 0.01, 'close': 0.01, 'energy_cost': 0.01, 'water_cost': 0.02, 'gamma': 0.9}, homeostasis={'energy': 3, 'water': 2, 'temp': 11, 'hormones': 10}, min_homeostasis={'energy': 10, 'water': 10, 'temp': 30, 'hormones': 0}, max_homeostasis={'energy': 10, 'water': 10, 'temp': 40, 'hormones': 10}, homeostatic_effects={('eat', 'dandelion'): {'energy': 1, 'water': 0.2}, ('drink', 'water'): {'energy': 0, 'water': 1}}, distances={'wolf': 100, 'dandelion': 2}, perception={'wolf': Perception(radius=10, origin=array([0., 0.]), steven_exponent=0.6, reward_radius=0.0, epsilon=0.31622776601683794, use_clip=False, direction_noise=0), 'dandelion': Perception(radius=2, origin=array([0., 0.]), steven_exponent=0.6, reward_radius=0.0, epsilon=0.31622776601683794, use_clip=False, direction_noise=0)}, interoception={'energy': 3, 'water': 2}, exteroception={'temp': 10, 'humid

In [ ]:
# Object perception

def object_perception(animal: Animal, ecosystem: Ecosystem, kind: str):
    positions = [organism.position for organism in [ecosystem.agent]]
    radius = animal.distances[kind]
    p = Perception(radius, animal.position, steven_exponent=0.6)
    visible = p.visible(animal.position, positions, radius)
    return p.multi_direction(animal.position, visible)

if IN_MAIN:
    object_perception(shaun,eco, 'wolf')

NameError: name 'distance' is not defined

In [ ]:
def update(ecosystem):
    # new = ecosystem
    # new.terrain = new.terrain # Maybe add weather and light conditions. Time...
    # new.organisms.positions, homeostasis, perception)
    # new terrain
    # return new
    pass1

## Discussion

## Comments

* What is Sheep doing now?
* Replace SimpleSheep with Moth?
* MultiSheep handles n object types and automatic consumption

### Improvements?

* Turn alive into a concrete class function in Organism

### Next step?

* Moth. Taxis can be based on object perception, e.g. a moth that is attracted to light bulbs. Happiness = total_intensity. The moth stays on the light bulb until it disappears. Then it flies on. Receives direction info.

* Silverfish. Silverfish dislike light bulbs. Happiness = -total_intensity. Receives direction info. Moves away from the gradient

* Pill bug. Taxis can also be based on exteroception, e.g. a pill bug that is attracted to humidity. Happiness = humidity. Receives no direction info (but maybe from two antennae). But might escape in a direction, so still taxis. Add speed (or effort) in [0,1] as addition to action space.

* Cat (with speed output).
    * Add speed (or torque or effort or energy) in [0,1] to the action space
    * Add gradient_intensity to the object perception (type, direction, total_intensity, gradient_intensity)

* Social sheep. 
    * Happiness = energy (0.6 + 0.4 oxytocin) 

* Modest sheep. 
    * Happiness function with diminishing return?


In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted animal_classes.ipynb.
Converted animation_helper.ipynb.
Converted ecoenv.ipynb.
Converted happiness.ipynb.
Converted index.ipynb.
Converted perception.ipynb.
Converted reflex_agent.ipynb.
Converted tyckande.ipynb.
Converted utility.ipynb.
Converted worlds.ipynb.
